In [1]:
import numpy as np
import pandas as pd

In [2]:
temp_df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df = temp_df.iloc[:10000]

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.drop_duplicates(inplace=True)

C:\Users\harsh\AppData\Local\Temp\ipykernel_9252\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [7]:
df['review'] = df['review'].apply(remove_tags)

C:\Users\harsh\AppData\Local\Temp\ipykernel_9252\2336150696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [8]:
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\harsh\AppData\Local\Temp\ipykernel_9252\740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [9]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\harsh\AppData\Local\Temp\ipykernel_9252\2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [10]:
df['review']

0       one reviewers mentioned watching 1 oz episode ...
1       wonderful little production. filming technique...
2       thought wonderful way spend time hot summer we...
3       basically there's family little boy (jake) thi...
4       petter mattei's "love time money" visually stu...
                              ...                        
9995    fun, entertaining movie wwii german spy (julie...
9996    give break. anyone say "good hockey movie"? kn...
9997    movie bad movie. watching endless series bad h...
9998    movie probably made entertain middle school, e...
9999    smashing film film-making. shows intense stran...
Name: review, Length: 9983, dtype: object

In [11]:
import gensim

In [12]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [13]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [14]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [15]:
model.build_vocab(story)

In [16]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5875855, 6212140)

In [17]:
len(model.wv.index_to_key)

31845

In [18]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [19]:
document_vector(df['review'].values[0])

array([-0.14405991,  0.4753924 ,  0.21885769,  0.247709  , -0.12411734,
       -0.57490337,  0.23629245,  0.92374235, -0.3882055 , -0.203664  ,
       -0.26043758, -0.5050433 ,  0.11763524,  0.10616158,  0.15719116,
       -0.12792681,  0.02155484, -0.35025644, -0.04644175, -0.6637647 ,
       -0.03927085,  0.17998278,  0.10256745, -0.281524  , -0.2781632 ,
       -0.04272392, -0.27031758,  0.04340788, -0.3034622 ,  0.03847469,
        0.33101758,  0.03995014,  0.18818824, -0.21271074, -0.16351616,
        0.45475325,  0.1591111 , -0.41268143, -0.22475383, -0.7144462 ,
        0.19354731, -0.22291653,  0.03018264, -0.08766796,  0.5331351 ,
       -0.09063644, -0.23803812, -0.01406765,  0.05129816,  0.31000108,
        0.05136228, -0.31437606, -0.4311697 , -0.12574424, -0.14887995,
        0.2562551 ,  0.20084918,  0.04687511, -0.2691473 ,  0.14043982,
        0.07197934,  0.13193156, -0.01901422, -0.08446985, -0.42994186,
        0.27046537,  0.05259455,  0.15218554, -0.42155474,  0.35

In [20]:
from tqdm import tqdm

In [21]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████████| 9983/9983 [22:39<00:00,  7.34it/s]


In [22]:
X = np.array(X)

In [23]:
X[0]

array([-0.14405991,  0.4753924 ,  0.21885769,  0.247709  , -0.12411734,
       -0.57490337,  0.23629245,  0.92374235, -0.3882055 , -0.203664  ,
       -0.26043758, -0.5050433 ,  0.11763524,  0.10616158,  0.15719116,
       -0.12792681,  0.02155484, -0.35025644, -0.04644175, -0.6637647 ,
       -0.03927085,  0.17998278,  0.10256745, -0.281524  , -0.2781632 ,
       -0.04272392, -0.27031758,  0.04340788, -0.3034622 ,  0.03847469,
        0.33101758,  0.03995014,  0.18818824, -0.21271074, -0.16351616,
        0.45475325,  0.1591111 , -0.41268143, -0.22475383, -0.7144462 ,
        0.19354731, -0.22291653,  0.03018264, -0.08766796,  0.5331351 ,
       -0.09063644, -0.23803812, -0.01406765,  0.05129816,  0.31000108,
        0.05136228, -0.31437606, -0.4311697 , -0.12574424, -0.14887995,
        0.2562551 ,  0.20084918,  0.04687511, -0.2691473 ,  0.14043982,
        0.07197934,  0.13193156, -0.01901422, -0.08446985, -0.42994186,
        0.27046537,  0.05259455,  0.15218554, -0.42155474,  0.35

In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [25]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [28]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7646469704556835